# FirstBar Farming - Professional Hedge Fund Analysis

## Executive Summary

This notebook implements a **comprehensive First Bar Trading Strategy analysis** with professional hedge fund-grade metrics, benchmarking, and reporting capabilities.

### Key Features:
- **Multi-Asset Strategy Testing**: Easy ticker switching (TQQQ, QQQ, SPY, etc.)
- **Comprehensive Benchmarking**: Against QQQ, TQQQ, and SPY buy-and-hold strategies
- **Advanced Risk Metrics**: Sortino ratio, Sharpe ratio, maximum drawdown analysis
- **Cash Analysis**: $100,000 portfolio simulation with realistic position sizing
- **Professional Reporting**: Executive summaries, methodology, and recommendations
- **IBKR Integration**: Live data from Interactive Brokers TWS API

### Strategy Logic:
- **Signal Generation**: First hourly bar direction determines trade (bullish/bearish momentum)
- **Entry**: Close of first trading hour (15:30 ET)
- **Exit**: Profit target achievement OR end-of-day close
- **Risk Management**: Same-day exits eliminate overnight gap risk

---

## 🔧 Professional Configuration System

In [ ]:
# ===================================================================
# PROFESSIONAL TRADING SYSTEM CONFIGURATION
# ===================================================================

# === PRIMARY STRATEGY CONFIGURATION ===
STRATEGY_TICKER = "TQQQ"        # Primary ticker for strategy testing
BENCHMARK_TICKERS = ["QQQ", "TQQQ", "SPY"]  # Benchmark comparison assets
STRATEGY_MODE = "both"          # "buy", "sell", "both"
PROFIT_TARGET = 2.2             # Optimal profit target (from QQQ analysis)
SMA_FILTER = 0                  # SMA filter period (0 = disabled)

# === PORTFOLIO SIMULATION SETTINGS ===
INITIAL_CAPITAL = 100000        # Starting portfolio value ($100,000)
POSITION_SIZE_PCT = 1.0         # Position size as % of portfolio (1% = conservative)
TRANSACTION_COST_PER_SHARE = 0.005  # Transaction cost per share ($0.005)
SLIPPAGE_BPS = 2                # Slippage in basis points (2 bps = 0.02%)

# === IBKR CONNECTION SETTINGS ===
TWS_HOST = "127.0.0.1"
TWS_PORT = 7497                 # 7497 = Paper, 7496 = Live
CLIENT_ID = 2                   # Unique client ID for this analysis

# === DATA CONFIGURATION ===
DATA_PERIOD = "3 Y"             # Historical data period
BAR_SIZE = "1 hour"             # Bar size for analysis
EXCHANGE = "SMART"              # Smart routing
CURRENCY = "USD"

# === REPORTING SETTINGS ===
REPORT_TITLE = "FirstBar Farming Strategy Analysis"
ANALYST_NAME = "FirstBar Quantitative Research"
REPORT_DATE = "2025-09-25"

print(f"🏛️ {REPORT_TITLE} - Configuration Loaded")
print(f"📊 Strategy Asset: {STRATEGY_TICKER} | Benchmarks: {', '.join(BENCHMARK_TICKERS)}")
print(f"💰 Portfolio Simulation: ${INITIAL_CAPITAL:,} initial capital")
print(f"🎯 Position Sizing: {POSITION_SIZE_PCT}% per trade")
print(f"📈 Profit Target: {PROFIT_TARGET}%")
print(f"🔌 IBKR Connection: {TWS_HOST}:{TWS_PORT}")

## 📚 Professional Imports and Setup

In [ ]:
# === CORE LIBRARIES ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from datetime import datetime, timedelta
import asyncio
import time
from typing import Dict, List, Tuple, Optional

# === STATISTICAL LIBRARIES ===
from scipy import stats
from scipy.stats import skew, kurtosis
import matplotlib.dates as mdates
from matplotlib.backends.backend_pdf import PdfPages

# === JUPYTER NOTEBOOK OPTIMIZATION ===
import nest_asyncio
nest_asyncio.apply()

# === IBKR INTEGRATION ===
try:
    from ib_insync import *
    IBKR_AVAILABLE = True
    print("✅ ib_insync imported successfully")
except ImportError:
    IBKR_AVAILABLE = False
    print("❌ ib_insync not available - limited functionality")

# === DISPLAY CONFIGURATION ===
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# === GLOBAL CONSTANTS ===
TRADING_DAYS_PER_YEAR = 252
RISK_FREE_RATE = 0.02  # 2% risk-free rate

print("📦 Professional libraries loaded successfully")
print(f"🕒 Analysis started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🎯 Jupyter asyncio optimization applied")

## 🏗️ Professional Data Management System

In [ ]:
class ProfessionalDataManager:
    """Professional-grade data management system for multi-asset analysis with automatic CSV export."""
    
    def __init__(self, host: str = TWS_HOST, port: int = TWS_PORT, client_id: int = CLIENT_ID):
        self.host = host
        self.port = port
        self.client_id = client_id
        self.ib = None
        self.connected = False
        self.data_cache = {}
        
    def connect(self) -> bool:
        """Establish professional IBKR connection with enhanced error handling."""
        if not IBKR_AVAILABLE:
            print("❌ IBKR not available - cannot establish connection")
            return False
            
        try:
            print(f"🔌 Establishing IBKR connection to {self.host}:{self.port}...")
            
            self.ib = IB()
            self.ib.connect(self.host, self.port, clientId=self.client_id, timeout=15)
            self.ib.sleep(2)  # Allow connection stabilization
            
            self.connected = True
            print(f"✅ Professional connection established (Client ID: {self.client_id})")
            
            # Validate connection with account information
            try:
                accounts = self.ib.managedAccounts()
                if accounts:
                    print(f"🏦 Connected to {len(accounts)} account(s)")
                    print(f"📊 Market data permissions verified")
                else:
                    print("⚠️ No account information available (may be normal for some setups)")
            except Exception as e:
                print(f"⚠️ Account info unavailable: {str(e)[:100]}")
            
            return True
            
        except Exception as e:
            print(f"❌ Connection failed: {e}")
            print("💡 Professional troubleshooting checklist:")
            print("   1. TWS/IB Gateway running and API enabled")
            print("   2. Correct port (7497=Paper, 7496=Live)")
            print("   3. Client ID not in use")
            print("   4. Market data subscriptions active")
            print("   5. Firewall settings allow connection")
            self.connected = False
            return False
    
    def create_professional_contract(self, symbol: str) -> Contract:
        """Create professionally configured IBKR contract."""
        # ETF and major equity configurations
        major_etfs = {
            'QQQ': ('STK', 'NASDAQ', 'USD'),
            'TQQQ': ('STK', 'NASDAQ', 'USD'),
            'SPY': ('STK', 'ARCA', 'USD'),
            'IWM': ('STK', 'ARCA', 'USD'),
            'DIA': ('STK', 'ARCA', 'USD')
        }
        
        if symbol.upper() in major_etfs:
            sec_type, exchange, currency = major_etfs[symbol.upper()]
            contract = Stock(symbol.upper(), exchange, currency)
        else:
            # Default configuration
            contract = Stock(symbol.upper(), EXCHANGE, CURRENCY)
        
        return contract
    
    def _export_to_csv(self, df: pd.DataFrame, symbol: str, duration: str, bar_size: str) -> str:
        """Export DataFrame to CSV with professional naming convention."""
        try:
            # Create filename with timestamp and parameters
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            duration_clean = duration.replace(' ', '').lower()
            bar_clean = bar_size.replace(' ', '_').lower()
            
            filename = f"../data/ibkr_{symbol.lower()}_{duration_clean}_{bar_clean}_{timestamp}.csv"
            
            # Reset index to include datetime as column for CSV export
            export_df = df.reset_index()
            
            # Save to CSV
            export_df.to_csv(filename, index=False)
            print(f"💾 Data exported to CSV: {filename}")
            
            return filename
            
        except Exception as e:
            print(f"⚠️ CSV export failed for {symbol}: {e}")
            return ""
    
    def fetch_professional_data(self, symbol: str, duration: str = DATA_PERIOD,
                              bar_size: str = BAR_SIZE, save_csv: bool = True) -> Optional[pd.DataFrame]:
        """Fetch professional-grade historical data with comprehensive validation and automatic CSV export."""
        if not self.connected:
            print(f"❌ Not connected to IBKR for {symbol}")
            return None

        try:
            print(f"📡 Fetching professional data for {symbol}...")

            # Create and qualify contract
            contract = self.create_professional_contract(symbol)
            qualified = self.ib.qualifyContracts(contract)

            if not qualified:
                print(f"❌ Contract qualification failed for {symbol}")
                return None

            contract = qualified[0]
            print(f"✅ Contract qualified: {contract.symbol} {contract.primaryExchange or contract.exchange}")

            # Request historical data with professional settings
            bars = self.ib.reqHistoricalData(
                contract,
                endDateTime='',
                durationStr=duration,
                barSizeSetting=bar_size,
                whatToShow='TRADES',
                useRTH=True,  # Regular trading hours
                formatDate=1,
                keepUpToDate=False
            )

            if not bars:
                print(f"❌ No data received for {symbol}")
                return None

            print(f"✅ Received {len(bars):,} bars for {symbol}")

            # Convert to professional DataFrame
            df = util.df(bars)

            if df.empty:
                print(f"❌ DataFrame conversion failed for {symbol}")
                return None

            # Professional data processing
            df = self._process_professional_data(df, symbol)

            # Automatic CSV export for transferability
            if save_csv:
                csv_file = self._export_to_csv(df, symbol, duration, bar_size)
                if csv_file:
                    print(f"📁 Data saved for transferability and reuse")

            # Cache data for efficiency
            self.data_cache[symbol] = df.copy()

            print(f"📊 {symbol} data processed: {len(df)} bars")
            print(f"📅 Period: {df.index[0].date()} to {df.index[-1].date()}")
            print(f"💰 Price range: ${df['close'].min():.2f} - ${df['close'].max():.2f}")

            return df

        except Exception as e:
            print(f"❌ Data fetch failed for {symbol}: {e}")
            return None
    
    def _process_professional_data(self, df: pd.DataFrame, symbol: str) -> pd.DataFrame:
        """Process data with professional standards."""
        # Standardize columns
        if 'date' in df.columns:
            df = df.rename(columns={'date': 'datetime'})
        
        # Ensure datetime index
        if 'datetime' in df.columns:
            df['datetime'] = pd.to_datetime(df['datetime'])
            df = df.set_index('datetime')
        
        # Ensure required columns exist
        required_cols = ['open', 'high', 'low', 'close', 'volume']
        for col in required_cols:
            if col not in df.columns:
                print(f"⚠️ Missing column {col} for {symbol}")
        
        # Add professional calculations
        df['returns'] = df['close'].pct_change()
        df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
        df['date'] = df.index.date
        df['symbol'] = symbol
        
        # Data quality checks
        df = df.dropna()
        
        return df
    
    def fetch_multiple_assets(self, symbols: List[str], save_csv: bool = True) -> Dict[str, pd.DataFrame]:
        """Fetch data for multiple assets efficiently with automatic CSV export."""
        results = {}
        
        for symbol in symbols:
            print(f"\n{'='*60}")
            print(f"Processing {symbol}...")
            print(f"{'='*60}")
            
            data = self.fetch_professional_data(symbol, save_csv=save_csv)
            if data is not None:
                results[symbol] = data
                time.sleep(1)  # Respect API limits
            else:
                print(f"⚠️ Skipping {symbol} due to data fetch failure")
        
        return results
    
    def load_from_csv(self, symbol: str, csv_file: str = None) -> Optional[pd.DataFrame]:
        """Load previously saved CSV data for reuse."""
        try:
            if csv_file is None:
                # Find most recent CSV file for symbol
                import glob
                pattern = f"../data/ibkr_{symbol.lower()}_*.csv"
                files = glob.glob(pattern)
                if not files:
                    print(f"❌ No CSV files found for {symbol}")
                    return None
                csv_file = max(files, key=lambda x: x.split('_')[-1])  # Most recent timestamp
            
            print(f"📁 Loading {symbol} data from CSV: {csv_file}")
            df = pd.read_csv(csv_file)
            
            # Process loaded data
            df['datetime'] = pd.to_datetime(df['datetime'])
            df = df.set_index('datetime')
            df['date'] = df.index.date
            
            print(f"✅ Loaded {len(df)} bars from CSV for {symbol}")
            print(f"📅 Period: {df.index[0].date()} to {df.index[-1].date()}")
            
            return df
            
        except Exception as e:
            print(f"❌ Failed to load CSV for {symbol}: {e}")
            return None
    
    def list_saved_data(self) -> List[str]:
        """List all saved CSV data files."""
        import glob
        files = glob.glob("../data/ibkr_*.csv")
        return sorted(files)
    
    def disconnect(self):
        """Professional disconnection with cleanup."""
        if self.ib and self.connected:
            try:
                self.ib.disconnect()
                print("🔌 Professional disconnection completed")
            except Exception as e:
                print(f"⚠️ Disconnect warning: {e}")
            finally:
                self.connected = False
                self.ib = None

# Initialize professional data manager
data_manager = ProfessionalDataManager()
print("🏗️ Professional Data Management System initialized with CSV export capability")
print("💾 All IBKR data will be automatically saved to CSV for transferability and reuse")

## 📊 Advanced Performance Analytics Engine

In [ ]:
class AdvancedPerformanceAnalytics:
    """Professional-grade performance analytics with hedge fund metrics."""
    
    @staticmethod
    def calculate_returns_metrics(returns: pd.Series) -> Dict:
        """Calculate comprehensive return metrics."""
        returns_clean = returns.dropna()
        
        if len(returns_clean) == 0:
            return {}
        
        # Basic metrics
        total_return = (1 + returns_clean).prod() - 1
        annualized_return = (1 + total_return) ** (TRADING_DAYS_PER_YEAR / len(returns_clean)) - 1
        volatility = returns_clean.std() * np.sqrt(TRADING_DAYS_PER_YEAR)
        
        # Risk metrics
        downside_returns = returns_clean[returns_clean < 0]
        downside_std = downside_returns.std() * np.sqrt(TRADING_DAYS_PER_YEAR) if len(downside_returns) > 0 else 0
        
        # Advanced ratios
        sharpe_ratio = (annualized_return - RISK_FREE_RATE) / volatility if volatility > 0 else 0
        sortino_ratio = (annualized_return - RISK_FREE_RATE) / downside_std if downside_std > 0 else 0
        
        # Distribution metrics
        skewness = skew(returns_clean) if len(returns_clean) > 2 else 0
        kurt = kurtosis(returns_clean) if len(returns_clean) > 3 else 0
        
        # Max drawdown calculation
        cumulative = (1 + returns_clean).cumprod()
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        max_drawdown = drawdown.min()
        
        # Calmar ratio
        calmar_ratio = annualized_return / abs(max_drawdown) if max_drawdown != 0 else 0
        
        return {
            'total_return': total_return,
            'annualized_return': annualized_return,
            'volatility': volatility,
            'sharpe_ratio': sharpe_ratio,
            'sortino_ratio': sortino_ratio,
            'max_drawdown': max_drawdown,
            'calmar_ratio': calmar_ratio,
            'skewness': skewness,
            'kurtosis': kurt,
            'win_rate': len(returns_clean[returns_clean > 0]) / len(returns_clean),
            'avg_win': returns_clean[returns_clean > 0].mean() if len(returns_clean[returns_clean > 0]) > 0 else 0,
            'avg_loss': returns_clean[returns_clean < 0].mean() if len(returns_clean[returns_clean < 0]) > 0 else 0,
            'profit_factor': abs(returns_clean[returns_clean > 0].sum() / returns_clean[returns_clean < 0].sum()) if returns_clean[returns_clean < 0].sum() != 0 else float('inf')
        }
    
    @staticmethod
    def calculate_portfolio_metrics(equity_curve: pd.Series, initial_capital: float = INITIAL_CAPITAL) -> Dict:
        """Calculate portfolio-level performance metrics."""
        if len(equity_curve) == 0:
            return {}
        
        # Portfolio value progression
        portfolio_values = initial_capital + equity_curve
        portfolio_returns = portfolio_values.pct_change().dropna()
        
        # Final portfolio value
        final_value = portfolio_values.iloc[-1]
        total_pnl = final_value - initial_capital
        
        # Performance metrics
        metrics = AdvancedPerformanceAnalytics.calculate_returns_metrics(portfolio_returns)
        
        # Additional portfolio metrics
        metrics.update({
            'initial_capital': initial_capital,
            'final_value': final_value,
            'total_pnl': total_pnl,
            'total_return_pct': (total_pnl / initial_capital) * 100
        })
        
        return metrics
    
    @staticmethod
    def create_benchmark_comparison(strategy_metrics: Dict, benchmark_metrics: Dict[str, Dict]) -> pd.DataFrame:
        """Create professional benchmark comparison table."""
        comparison_data = {'FirstBar Strategy': strategy_metrics}
        comparison_data.update(benchmark_metrics)
        
        df = pd.DataFrame(comparison_data).T
        
        # Select key metrics for comparison
        key_metrics = [
            'total_return_pct', 'annualized_return', 'volatility',
            'sharpe_ratio', 'sortino_ratio', 'max_drawdown',
            'calmar_ratio', 'final_value'
        ]
        
        # Filter to available metrics
        available_metrics = [m for m in key_metrics if m in df.columns]
        
        return df[available_metrics]

# Initialize analytics engine
analytics = AdvancedPerformanceAnalytics()
print("📊 Advanced Performance Analytics Engine initialized")

## 🎯 Professional FirstBar Strategy Engine

In [ ]:
class ProfessionalFirstBarStrategy:
    """Professional implementation of the FirstBar trading strategy."""
    
    def __init__(self, initial_capital: float = INITIAL_CAPITAL, 
                 position_size_pct: float = POSITION_SIZE_PCT,
                 transaction_cost: float = TRANSACTION_COST_PER_SHARE,
                 slippage_bps: float = SLIPPAGE_BPS):
        self.initial_capital = initial_capital
        self.position_size_pct = position_size_pct
        self.transaction_cost = transaction_cost
        self.slippage_bps = slippage_bps
    
    def identify_first_bars(self, data: pd.DataFrame) -> pd.DataFrame:
        """Identify first trading bar of each day."""
        if 'date' not in data.columns:
            data['date'] = data.index.date
        
        first_bars = data.groupby('date').first().reset_index()
        first_bars = first_bars.set_index(data.index.name or 'datetime')
        
        return first_bars
    
    def generate_signals(self, first_bars: pd.DataFrame, mode: str = STRATEGY_MODE) -> pd.DataFrame:
        """Generate trading signals from first bars."""
        signals = first_bars.copy()
        
        # Basic momentum signals
        signals['signal'] = np.where(
            signals['close'] > signals['open'], 'BUY',
            np.where(signals['close'] < signals['open'], 'SELL', 'NONE')
        )
        
        # Apply strategy mode filter
        if mode == 'buy':
            signals.loc[signals['signal'] == 'SELL', 'signal'] = 'NONE'
        elif mode == 'sell':
            signals.loc[signals['signal'] == 'BUY', 'signal'] = 'NONE'
        
        return signals[signals['signal'] != 'NONE']
    
    def simulate_trades(self, data: pd.DataFrame, signals: pd.DataFrame, 
                       profit_target: float = PROFIT_TARGET) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Simulate trades with professional execution model."""
        trades = []
        current_capital = self.initial_capital
        
        for signal_time, signal_row in signals.iterrows():
            # Get same-day data for exit simulation
            signal_date = signal_row['date']
            same_day_data = data[data['date'] == signal_date].copy()
            
            if len(same_day_data) == 0:
                continue
            
            # Entry details
            entry_price = signal_row['close']
            signal_type = signal_row['signal']
            
            # Calculate position size based on current capital
            position_value = current_capital * (self.position_size_pct / 100)
            shares = int(position_value / entry_price)
            
            if shares <= 0:
                continue
            
            # Apply slippage to entry
            slippage_factor = 1 + (self.slippage_bps / 10000)
            if signal_type == 'BUY':
                actual_entry_price = entry_price * slippage_factor
            else:
                actual_entry_price = entry_price / slippage_factor
            
            # Find exit point
            exit_time, exit_price = self._find_exit_point(
                same_day_data, signal_time, signal_type, 
                actual_entry_price, profit_target
            )
            
            # Apply slippage to exit
            if signal_type == 'BUY':
                actual_exit_price = exit_price / slippage_factor
            else:
                actual_exit_price = exit_price * slippage_factor
            
            # Calculate P&L
            if signal_type == 'BUY':
                gross_pnl = (actual_exit_price - actual_entry_price) * shares
            else:
                gross_pnl = (actual_entry_price - actual_exit_price) * shares
            
            # Apply transaction costs
            transaction_costs = 2 * shares * self.transaction_cost  # Entry + Exit
            net_pnl = gross_pnl - transaction_costs
            
            # Update capital
            current_capital += net_pnl
            
            # Record trade
            trades.append({
                'entry_time': signal_time,
                'exit_time': exit_time,
                'symbol': signal_row.get('symbol', 'Unknown'),
                'signal': signal_type,
                'shares': shares,
                'entry_price': actual_entry_price,
                'exit_price': actual_exit_price,
                'gross_pnl': gross_pnl,
                'transaction_costs': transaction_costs,
                'net_pnl': net_pnl,
                'portfolio_value': current_capital,
                'return_pct': (net_pnl / (actual_entry_price * shares)) * 100
            })
        
        trades_df = pd.DataFrame(trades)
        
        if trades_df.empty:
            return pd.DataFrame(), pd.DataFrame()
        
        # Create equity curve
        trades_df['cumulative_pnl'] = trades_df['net_pnl'].cumsum()
        
        equity_curve = pd.DataFrame({
            'time': trades_df['exit_time'],
            'portfolio_value': trades_df['portfolio_value'],
            'cumulative_pnl': trades_df['cumulative_pnl']
        }).set_index('time')
        
        return trades_df, equity_curve
    
    def _find_exit_point(self, same_day_data: pd.DataFrame, entry_time, signal_type: str, 
                        entry_price: float, profit_target: float) -> Tuple:
        """Find optimal exit point for trade."""
        # Get bars after entry
        after_entry = same_day_data[same_day_data.index > entry_time]
        
        if len(after_entry) == 0:
            # Exit at same bar if no future data
            return entry_time, entry_price
        
        # Check for profit target hit
        if profit_target > 0:
            if signal_type == 'BUY':
                target_price = entry_price * (1 + profit_target / 100)
                hit_bars = after_entry[after_entry['high'] >= target_price]
                if len(hit_bars) > 0:
                    return hit_bars.index[0], target_price
            else:  # SELL
                target_price = entry_price * (1 - profit_target / 100)
                hit_bars = after_entry[after_entry['low'] <= target_price]
                if len(hit_bars) > 0:
                    return hit_bars.index[0], target_price
        
        # Exit at end of day
        last_bar = after_entry.iloc[-1]
        return after_entry.index[-1], last_bar['close']
    
    def create_benchmark_portfolio(self, data: pd.DataFrame, symbol: str) -> Tuple[pd.DataFrame, Dict]:
        """Create buy-and-hold benchmark portfolio."""
        if data.empty:
            return pd.DataFrame(), {}
        
        # Calculate number of shares that can be bought with initial capital
        initial_price = data['close'].iloc[0]
        shares = int(self.initial_capital / initial_price)
        actual_invested = shares * initial_price
        cash_remainder = self.initial_capital - actual_invested
        
        # Create portfolio value series
        portfolio_values = shares * data['close'] + cash_remainder
        portfolio_returns = portfolio_values.pct_change().dropna()
        
        # Calculate metrics
        final_value = portfolio_values.iloc[-1]
        total_pnl = final_value - self.initial_capital
        
        portfolio_df = pd.DataFrame({
            'portfolio_value': portfolio_values,
            'returns': portfolio_returns.reindex(data.index)
        })
        
        metrics = analytics.calculate_portfolio_metrics(
            portfolio_values - self.initial_capital, self.initial_capital
        )
        
        return portfolio_df, metrics

# Initialize strategy engine
strategy = ProfessionalFirstBarStrategy()
print("🎯 Professional FirstBar Strategy Engine initialized")
print(f"💰 Capital: ${INITIAL_CAPITAL:,} | Position Size: {POSITION_SIZE_PCT}%")
print(f"🔧 Transaction Cost: ${TRANSACTION_COST_PER_SHARE}/share | Slippage: {SLIPPAGE_BPS} bps")

## 📡 Data Acquisition and Processing

In [ ]:
# Establish IBKR connection and fetch data with automatic CSV export
print(f"🚀 Starting professional data acquisition for FirstBar analysis...")
print(f"{'='*80}")

# Connect to IBKR
if data_manager.connect():
    print(f"\n📡 Fetching data for strategy and benchmarks...")
    print(f"💾 All data will be automatically exported to CSV for transferability")
    
    # Get all required symbols
    all_symbols = [STRATEGY_TICKER] + [ticker for ticker in BENCHMARK_TICKERS if ticker != STRATEGY_TICKER]
    
    # Fetch all data with automatic CSV export
    market_data = data_manager.fetch_multiple_assets(all_symbols, save_csv=True)
    
    # Disconnect
    data_manager.disconnect()
    
    print(f"\n{'='*80}")
    print(f"📊 DATA ACQUISITION SUMMARY")
    print(f"{'='*80}")
    
    for symbol, data in market_data.items():
        print(f"✅ {symbol}: {len(data):,} bars ({data.index[0].date()} to {data.index[-1].date()})")
    
    if len(market_data) < len(all_symbols):
        missing = set(all_symbols) - set(market_data.keys())
        print(f"⚠️ Missing data for: {', '.join(missing)}")
    
    # Show saved CSV files
    saved_files = data_manager.list_saved_data()
    if saved_files:
        print(f"\n💾 CSV FILES SAVED FOR TRANSFERABILITY:")
        for file in saved_files[-len(market_data):]:  # Show recent files
            print(f"   📁 {file}")
    
else:
    print("❌ Failed to establish IBKR connection")
    print("💡 Please ensure TWS/IB Gateway is running with API enabled")
    print("💡 Alternative: Load from previously saved CSV files")
    market_data = {}

## 🎯 Strategy Execution and Analysis

In [ ]:
# Execute FirstBar Strategy
if STRATEGY_TICKER in market_data:
    strategy_data = market_data[STRATEGY_TICKER]
    
    print(f"\n🎯 Executing FirstBar Strategy on {STRATEGY_TICKER}...")
    print(f"{'='*80}")
    
    # Run strategy
    first_bars = strategy.identify_first_bars(strategy_data)
    signals = strategy.generate_signals(first_bars, STRATEGY_MODE)
    trades_df, equity_curve = strategy.simulate_trades(strategy_data, signals, PROFIT_TARGET)
    
    if not trades_df.empty:
        # Calculate strategy performance metrics
        strategy_metrics = analytics.calculate_portfolio_metrics(
            trades_df['cumulative_pnl'], INITIAL_CAPITAL
        )
        
        print(f"\n🏆 STRATEGY PERFORMANCE SUMMARY")
        print(f"{'='*50}")
        print(f"📊 Total Trades: {len(trades_df):,}")
        print(f"💰 Final Portfolio Value: ${strategy_metrics['final_value']:,.2f}")
        print(f"📈 Total Return: {strategy_metrics['total_return_pct']:.2f}%")
        print(f"📊 Annualized Return: {strategy_metrics['annualized_return']*100:.2f}%")
        print(f"📉 Maximum Drawdown: {strategy_metrics['max_drawdown']*100:.2f}%")
        print(f"⚡ Sharpe Ratio: {strategy_metrics['sharpe_ratio']:.3f}")
        print(f"🎯 Sortino Ratio: {strategy_metrics['sortino_ratio']:.3f}")
        print(f"🏆 Calmar Ratio: {strategy_metrics['calmar_ratio']:.3f}")
        
        strategy_success = True
    else:
        print("❌ No trades generated for strategy")
        strategy_success = False
else:
    print(f"❌ Strategy data not available for {STRATEGY_TICKER}")
    strategy_success = False

## 📊 Benchmark Analysis

In [ ]:
# Calculate benchmark performance
benchmark_results = {}
benchmark_portfolios = {}

print(f"\n📊 Analyzing benchmark performance...")
print(f"{'='*80}")

for ticker in BENCHMARK_TICKERS:
    if ticker in market_data:
        print(f"\n📈 Processing {ticker} benchmark...")
        
        benchmark_portfolio, benchmark_metrics = strategy.create_benchmark_portfolio(
            market_data[ticker], ticker
        )
        
        if benchmark_metrics:
            benchmark_results[f"{ticker} Buy-Hold"] = benchmark_metrics
            benchmark_portfolios[ticker] = benchmark_portfolio
            
            print(f"✅ {ticker} Analysis Complete:")
            print(f"   💰 Final Value: ${benchmark_metrics['final_value']:,.2f}")
            print(f"   📈 Total Return: {benchmark_metrics['total_return_pct']:.2f}%")
            print(f"   📊 Annualized Return: {benchmark_metrics['annualized_return']*100:.2f}%")
            print(f"   ⚡ Sharpe Ratio: {benchmark_metrics['sharpe_ratio']:.3f}")
        else:
            print(f"⚠️ {ticker} benchmark calculation failed")
    else:
        print(f"⚠️ {ticker} data not available for benchmarking")

print(f"\n✅ Benchmark analysis complete: {len(benchmark_results)} benchmarks processed")

## 📈 Professional Visualization Dashboard

In [ ]:
if strategy_success and benchmark_results:
    print("📊 Creating professional visualization dashboard...")
    
    # Create comprehensive dashboard
    fig = plt.figure(figsize=(20, 16))
    gs = fig.add_gridspec(4, 3, hspace=0.3, wspace=0.3)
    
    # 1. Portfolio Value Comparison (Large)
    ax1 = fig.add_subplot(gs[0, :])
    
    # Plot strategy equity curve
    strategy_portfolio = INITIAL_CAPITAL + trades_df['cumulative_pnl'].reindex(equity_curve.index, method='ffill')
    ax1.plot(equity_curve.index, strategy_portfolio, 
             linewidth=3, label=f'{STRATEGY_TICKER} FirstBar Strategy', color='#2E8B57')
    
    # Plot benchmarks
    colors = ['#4169E1', '#DC143C', '#FF8C00']
    for i, (ticker, portfolio) in enumerate(benchmark_portfolios.items()):
        ax1.plot(portfolio.index, portfolio['portfolio_value'], 
                linewidth=2, label=f'{ticker} Buy-Hold', color=colors[i % len(colors)], alpha=0.8)
    
    ax1.set_title('Portfolio Value Comparison ($100,000 Initial Capital)', fontsize=16, fontweight='bold')
    ax1.set_ylabel('Portfolio Value ($)', fontsize=12)
    ax1.legend(loc='upper left', fontsize=10)
    ax1.grid(True, alpha=0.3)
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    
    # 2. Performance Metrics Comparison
    ax2 = fig.add_subplot(gs[1, 0])
    
    if benchmark_results:
        comparison_df = analytics.create_benchmark_comparison(strategy_metrics, benchmark_results)
        
        # Bar chart of total returns
        returns_data = comparison_df['total_return_pct'].dropna()
        colors_bar = ['#2E8B57' if 'FirstBar' in x else '#4169E1' for x in returns_data.index]
        bars = ax2.bar(range(len(returns_data)), returns_data.values, color=colors_bar, alpha=0.8)
        ax2.set_title('Total Returns Comparison', fontsize=12, fontweight='bold')
        ax2.set_ylabel('Total Return (%)')
        ax2.set_xticks(range(len(returns_data)))
        ax2.set_xticklabels([x.replace(' Buy-Hold', '') for x in returns_data.index], rotation=45, ha='right')
        ax2.grid(True, alpha=0.3, axis='y')
        
        # Add value labels on bars
        for bar in bars:
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                    f'{height:.1f}%', ha='center', va='bottom', fontsize=9)
    
    # 3. Risk-Adjusted Returns (Sharpe Ratios)
    ax3 = fig.add_subplot(gs[1, 1])
    
    if benchmark_results:
        sharpe_data = comparison_df['sharpe_ratio'].dropna()
        colors_sharpe = ['#2E8B57' if 'FirstBar' in x else '#DC143C' for x in sharpe_data.index]
        bars = ax3.bar(range(len(sharpe_data)), sharpe_data.values, color=colors_sharpe, alpha=0.8)
        ax3.set_title('Sharpe Ratio Comparison', fontsize=12, fontweight='bold')
        ax3.set_ylabel('Sharpe Ratio')
        ax3.set_xticks(range(len(sharpe_data)))
        ax3.set_xticklabels([x.replace(' Buy-Hold', '') for x in sharpe_data.index], rotation=45, ha='right')
        ax3.grid(True, alpha=0.3, axis='y')
        
        # Add value labels
        for bar in bars:
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{height:.3f}', ha='center', va='bottom', fontsize=9)
    
    # 4. Maximum Drawdown Comparison
    ax4 = fig.add_subplot(gs[1, 2])
    
    if benchmark_results:
        dd_data = comparison_df['max_drawdown'].dropna() * 100  # Convert to percentage
        colors_dd = ['#2E8B57' if 'FirstBar' in x else '#FF8C00' for x in dd_data.index]
        bars = ax4.bar(range(len(dd_data)), dd_data.values, color=colors_dd, alpha=0.8)
        ax4.set_title('Maximum Drawdown', fontsize=12, fontweight='bold')
        ax4.set_ylabel('Max Drawdown (%)')
        ax4.set_xticks(range(len(dd_data)))
        ax4.set_xticklabels([x.replace(' Buy-Hold', '') for x in dd_data.index], rotation=45, ha='right')
        ax4.grid(True, alpha=0.3, axis='y')
        
        # Add value labels
        for bar in bars:
            height = bar.get_height()
            ax4.text(bar.get_x() + bar.get_width()/2., height - 0.5,
                    f'{height:.1f}%', ha='center', va='top', fontsize=9)
    
    # 5. Strategy Trade Distribution
    ax5 = fig.add_subplot(gs[2, 0])
    
    returns = trades_df['return_pct'].dropna()
    ax5.hist(returns, bins=30, alpha=0.7, color='#2E8B57', edgecolor='black')
    ax5.axvline(returns.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {returns.mean():.2f}%')
    ax5.set_title('Strategy Trade Return Distribution', fontsize=12, fontweight='bold')
    ax5.set_xlabel('Return per Trade (%)')
    ax5.set_ylabel('Frequency')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    # 6. Monthly Returns Heatmap
    ax6 = fig.add_subplot(gs[2, 1])
    
    # Prepare monthly returns data
    trades_monthly = trades_df.copy()
    trades_monthly['month'] = pd.to_datetime(trades_monthly['exit_time']).dt.to_period('M')
    monthly_returns = trades_monthly.groupby('month')['return_pct'].mean()
    
    if len(monthly_returns) > 12:
        # Show last 12 months if more data available
        monthly_returns = monthly_returns.tail(12)
    
    colors = ['red' if x < 0 else 'green' for x in monthly_returns.values]
    bars = ax6.bar(range(len(monthly_returns)), monthly_returns.values, color=colors, alpha=0.7)
    ax6.set_title('Monthly Average Returns', fontsize=12, fontweight='bold')
    ax6.set_ylabel('Avg Return (%)')
    ax6.set_xticks(range(len(monthly_returns)))
    ax6.set_xticklabels([str(x) for x in monthly_returns.index], rotation=45, ha='right')
    ax6.grid(True, alpha=0.3, axis='y')
    ax6.axhline(y=0, color='black', linestyle='-', alpha=0.5)
    
    # 7. Sortino Ratio Comparison
    ax7 = fig.add_subplot(gs[2, 2])
    
    if benchmark_results:
        sortino_data = comparison_df['sortino_ratio'].dropna()
        colors_sortino = ['#2E8B57' if 'FirstBar' in x else '#9370DB' for x in sortino_data.index]
        bars = ax7.bar(range(len(sortino_data)), sortino_data.values, color=colors_sortino, alpha=0.8)
        ax7.set_title('Sortino Ratio Comparison', fontsize=12, fontweight='bold')
        ax7.set_ylabel('Sortino Ratio')
        ax7.set_xticks(range(len(sortino_data)))
        ax7.set_xticklabels([x.replace(' Buy-Hold', '') for x in sortino_data.index], rotation=45, ha='right')
        ax7.grid(True, alpha=0.3, axis='y')
        
        # Add value labels
        for bar in bars:
            height = bar.get_height()
            ax7.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{height:.3f}', ha='center', va='bottom', fontsize=9)
    
    # 8. Performance Summary Table
    ax8 = fig.add_subplot(gs[3, :])
    ax8.axis('off')
    
    if benchmark_results:
        # Create summary table
        table_data = []
        metrics_to_show = ['final_value', 'total_return_pct', 'annualized_return', 'volatility', 
                          'sharpe_ratio', 'sortino_ratio', 'max_drawdown', 'calmar_ratio']
        
        for asset in comparison_df.index:
            row = [asset.replace(' Buy-Hold', '')]
            for metric in metrics_to_show:
                if metric in comparison_df.columns:
                    value = comparison_df.loc[asset, metric]
                    if metric == 'final_value':
                        row.append(f'${value:,.0f}')
                    elif metric in ['total_return_pct', 'annualized_return', 'volatility', 'max_drawdown']:
                        row.append(f'{value*100:.2f}%' if metric != 'total_return_pct' else f'{value:.2f}%')
                    else:
                        row.append(f'{value:.3f}')
                else:
                    row.append('N/A')
            table_data.append(row)
        
        headers = ['Asset', 'Final Value', 'Total Return', 'Ann. Return', 'Volatility', 
                  'Sharpe', 'Sortino', 'Max DD', 'Calmar']
        
        table = ax8.table(cellText=table_data, colLabels=headers, cellLoc='center', loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        
        # Style the table
        for i in range(len(headers)):
            table[(0, i)].set_facecolor('#4CAF50')
            table[(0, i)].set_text_props(weight='bold', color='white')
        
        for i in range(1, len(table_data) + 1):
            if 'FirstBar' in table_data[i-1][0]:
                for j in range(len(headers)):
                    table[(i, j)].set_facecolor('#E8F5E8')
    
    plt.suptitle(f'{REPORT_TITLE}\nComparative Analysis: {STRATEGY_TICKER} Strategy vs Benchmarks', 
                fontsize=18, fontweight='bold', y=0.98)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Professional visualization dashboard created")
    
else:
    print("⚠️ Insufficient data for visualization dashboard")

## 📋 Executive Performance Summary

In [ ]:
if strategy_success and benchmark_results:
    print(f"\n{'='*100}")
    print(f"🏛️ FIRSTBAR FARMING - EXECUTIVE PERFORMANCE SUMMARY")
    print(f"{'='*100}")
    
    print(f"\n📊 STRATEGY OVERVIEW:")
    print(f"   • Primary Asset: {STRATEGY_TICKER}")
    print(f"   • Strategy Mode: {STRATEGY_MODE.title()}")
    print(f"   • Profit Target: {PROFIT_TARGET}%")
    print(f"   • Initial Capital: ${INITIAL_CAPITAL:,}")
    print(f"   • Position Size: {POSITION_SIZE_PCT}% per trade")
    print(f"   • Analysis Period: {strategy_data.index[0].date()} to {strategy_data.index[-1].date()}")
    
    print(f"\n🎯 STRATEGY PERFORMANCE:")
    print(f"   • Total Trades: {len(trades_df):,}")
    print(f"   • Final Portfolio Value: ${strategy_metrics['final_value']:,.2f}")
    print(f"   • Total Profit: ${strategy_metrics['total_pnl']:,.2f}")
    print(f"   • Total Return: {strategy_metrics['total_return_pct']:.2f}%")
    print(f"   • Annualized Return: {strategy_metrics['annualized_return']*100:.2f}%")
    print(f"   • Volatility: {strategy_metrics['volatility']*100:.2f}%")
    
    print(f"\n⚡ RISK-ADJUSTED METRICS:")
    print(f"   • Sharpe Ratio: {strategy_metrics['sharpe_ratio']:.3f}")
    print(f"   • Sortino Ratio: {strategy_metrics['sortino_ratio']:.3f}")
    print(f"   • Calmar Ratio: {strategy_metrics['calmar_ratio']:.3f}")
    print(f"   • Maximum Drawdown: {strategy_metrics['max_drawdown']*100:.2f}%")
    
    print(f"\n📈 TRADE STATISTICS:")
    win_trades = trades_df[trades_df['net_pnl'] > 0]
    loss_trades = trades_df[trades_df['net_pnl'] < 0]
    print(f"   • Win Rate: {len(win_trades)/len(trades_df)*100:.1f}%")
    print(f"   • Average Win: ${win_trades['net_pnl'].mean():.2f}" if len(win_trades) > 0 else "   • Average Win: $0.00")
    print(f"   • Average Loss: ${loss_trades['net_pnl'].mean():.2f}" if len(loss_trades) > 0 else "   • Average Loss: $0.00")
    print(f"   • Largest Win: ${trades_df['net_pnl'].max():.2f}")
    print(f"   • Largest Loss: ${trades_df['net_pnl'].min():.2f}")
    
    if len(win_trades) > 0 and len(loss_trades) > 0:
        profit_factor = abs(win_trades['net_pnl'].sum() / loss_trades['net_pnl'].sum())
        print(f"   • Profit Factor: {profit_factor:.2f}")
    
    print(f"\n🏆 BENCHMARK COMPARISON:")
    comparison_df = analytics.create_benchmark_comparison(strategy_metrics, benchmark_results)
    
    # Rank by total return
    returns_ranking = comparison_df['total_return_pct'].sort_values(ascending=False)
    strategy_rank = list(returns_ranking.index).index('FirstBar Strategy') + 1
    
    print(f"   • Total Return Ranking: #{strategy_rank} out of {len(returns_ranking)}")
    
    for i, (asset, return_pct) in enumerate(returns_ranking.items(), 1):
        symbol = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else "  "
        name = asset.replace(' Buy-Hold', '')
        print(f"   {symbol} #{i}: {name} - {return_pct:.2f}%")
    
    # Risk-adjusted ranking
    if 'sharpe_ratio' in comparison_df.columns:
        sharpe_ranking = comparison_df['sharpe_ratio'].sort_values(ascending=False)
        strategy_sharpe_rank = list(sharpe_ranking.index).index('FirstBar Strategy') + 1
        print(f"\n   • Sharpe Ratio Ranking: #{strategy_sharpe_rank} out of {len(sharpe_ranking)}")
        
        for i, (asset, sharpe) in enumerate(sharpe_ranking.items(), 1):
            symbol = "⭐" if i == 1 else "  "
            name = asset.replace(' Buy-Hold', '')
            print(f"   {symbol} #{i}: {name} - {sharpe:.3f}")
    
    print(f"\n💡 KEY INSIGHTS:")
    
    # Performance insights
    best_benchmark = returns_ranking.index[0] if 'FirstBar Strategy' not in returns_ranking.index[0] else returns_ranking.index[1]
    best_return = returns_ranking.iloc[0] if 'FirstBar Strategy' not in returns_ranking.index[0] else returns_ranking.iloc[1]
    strategy_return = strategy_metrics['total_return_pct']
    
    if strategy_return > best_return:
        outperformance = strategy_return - best_return
        print(f"   • Strategy OUTPERFORMED best benchmark by {outperformance:.2f}%")
    else:
        underperformance = best_return - strategy_return
        print(f"   • Strategy underperformed {best_benchmark.replace(' Buy-Hold', '')} by {underperformance:.2f}%")
    
    # Risk insights
    if strategy_metrics['sharpe_ratio'] > 1.0:
        print(f"   • EXCELLENT risk-adjusted returns (Sharpe > 1.0)")
    elif strategy_metrics['sharpe_ratio'] > 0.5:
        print(f"   • GOOD risk-adjusted returns (Sharpe > 0.5)")
    else:
        print(f"   • Risk-adjusted returns need improvement (Sharpe < 0.5)")
    
    # Drawdown insights
    if abs(strategy_metrics['max_drawdown']) < 0.15:
        print(f"   • EXCELLENT drawdown control (< 15%)")
    elif abs(strategy_metrics['max_drawdown']) < 0.25:
        print(f"   • GOOD drawdown management (< 25%)")
    else:
        print(f"   • High drawdown risk (> 25%) - consider risk management")
    
    print(f"\n📄 PROFESSIONAL RECOMMENDATION:")
    
    if strategy_rank <= 2 and strategy_metrics['sharpe_ratio'] > 0.8:
        print(f"   🟢 STRONG BUY: Excellent performance with superior risk-adjusted returns")
    elif strategy_rank <= 3 and strategy_metrics['sharpe_ratio'] > 0.5:
        print(f"   🟡 MODERATE BUY: Good performance, monitor risk management")
    elif strategy_metrics['sharpe_ratio'] > 0.3:
        print(f"   🟡 HOLD: Acceptable performance, consider optimization")
    else:
        print(f"   🔴 REVIEW: Performance needs improvement, reassess parameters")
    
    print(f"\n{'='*100}")
    print(f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Analyst: {ANALYST_NAME}")
    print(f"{'='*100}")
    
else:
    print("⚠️ Executive summary not available - insufficient data")

## 💾 Data Export and Archival

In [ ]:
if strategy_success:
    print("💾 Exporting professional analysis results...")
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Export strategy trades
    trades_file = f"../results/firstbar_strategy_trades_{STRATEGY_TICKER.lower()}_{timestamp}.csv"
    trades_df.to_csv(trades_file, index=False)
    print(f"✅ Strategy trades exported: {trades_file}")
    
    # Export equity curve
    equity_file = f"../results/firstbar_equity_curve_{STRATEGY_TICKER.lower()}_{timestamp}.csv"
    equity_curve.to_csv(equity_file)
    print(f"✅ Equity curve exported: {equity_file}")
    
    # Export performance comparison
    if benchmark_results:
        comparison_file = f"../results/benchmark_comparison_{STRATEGY_TICKER.lower()}_{timestamp}.csv"
        comparison_df.to_csv(comparison_file)
        print(f"✅ Benchmark comparison exported: {comparison_file}")
    
    # Export summary metrics
    summary_data = {
        'Analysis_Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'Strategy_Ticker': STRATEGY_TICKER,
        'Strategy_Mode': STRATEGY_MODE,
        'Profit_Target': PROFIT_TARGET,
        'Initial_Capital': INITIAL_CAPITAL,
        'Final_Value': strategy_metrics['final_value'],
        'Total_Return_Pct': strategy_metrics['total_return_pct'],
        'Annualized_Return': strategy_metrics['annualized_return'],
        'Sharpe_Ratio': strategy_metrics['sharpe_ratio'],
        'Sortino_Ratio': strategy_metrics['sortino_ratio'],
        'Max_Drawdown': strategy_metrics['max_drawdown'],
        'Calmar_Ratio': strategy_metrics['calmar_ratio'],
        'Total_Trades': len(trades_df)
    }
    
    summary_df = pd.DataFrame([summary_data])
    summary_file = f"../results/strategy_summary_{STRATEGY_TICKER.lower()}_{timestamp}.csv"
    summary_df.to_csv(summary_file, index=False)
    print(f"✅ Strategy summary exported: {summary_file}")
    
    print(f"\n📁 All results saved to: ../results/")
    print(f"🏷️ Timestamp: {timestamp}")
    
else:
    print("⚠️ No data to export - strategy execution unsuccessful")

## 📄 Professional PDF Report Generation

In [ ]:
# Note: PDF generation requires additional setup
# For now, we'll create a comprehensive text report

if strategy_success and benchmark_results:
    print("📄 Generating professional analysis report...")
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    report_file = f"../reports/FirstBar_Professional_Report_{STRATEGY_TICKER}_{timestamp}.txt"
    
    with open(report_file, 'w') as f:
        f.write("="*100 + "\n")
        f.write(f"FIRSTBAR FARMING - PROFESSIONAL HEDGE FUND ANALYSIS\n")
        f.write("="*100 + "\n")
        f.write(f"\nReport Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Analyst: {ANALYST_NAME}\n")
        f.write(f"Strategy: FirstBar Momentum Trading\n")
        f.write(f"Primary Asset: {STRATEGY_TICKER}\n")
        f.write(f"Analysis Period: {strategy_data.index[0].date()} to {strategy_data.index[-1].date()}\n")
        
        f.write("\n" + "="*50 + "\n")
        f.write("EXECUTIVE SUMMARY\n")
        f.write("="*50 + "\n")
        
        f.write(f"\nThe FirstBar Strategy generated a total return of {strategy_metrics['total_return_pct']:.2f}% ")
        f.write(f"over the analysis period, compared to {STRATEGY_TICKER} buy-and-hold. ")
        
        if STRATEGY_TICKER in benchmark_results:
            benchmark_return = benchmark_results[f'{STRATEGY_TICKER} Buy-Hold']['total_return_pct']
            if strategy_metrics['total_return_pct'] > benchmark_return:
                outperformance = strategy_metrics['total_return_pct'] - benchmark_return
                f.write(f"The strategy OUTPERFORMED the benchmark by {outperformance:.2f}%. ")
            else:
                underperformance = benchmark_return - strategy_metrics['total_return_pct']
                f.write(f"The strategy underperformed the benchmark by {underperformance:.2f}%. ")
        
        f.write(f"\n\nRisk-adjusted performance shows a Sharpe ratio of {strategy_metrics['sharpe_ratio']:.3f} ")
        f.write(f"and a Sortino ratio of {strategy_metrics['sortino_ratio']:.3f}, indicating ")
        
        if strategy_metrics['sharpe_ratio'] > 1.0:
            f.write("excellent risk-adjusted returns.")
        elif strategy_metrics['sharpe_ratio'] > 0.5:
            f.write("good risk-adjusted returns.")
        else:
            f.write("room for improvement in risk management.")
        
        f.write("\n" + "="*50 + "\n")
        f.write("METHODOLOGY\n")
        f.write("="*50 + "\n")
        
        f.write("\nThe FirstBar Strategy is based on momentum trading principles:\n")
        f.write("• Signal Generation: Direction of first hourly trading bar determines trade\n")
        f.write("• Entry: Close of first trading hour (15:30 ET)\n")
        f.write("• Exit: Profit target achievement OR end-of-day close\n")
        f.write("• Risk Management: Same-day exits eliminate overnight gap risk\n")
        f.write(f"• Position Sizing: {POSITION_SIZE_PCT}% of portfolio per trade\n")
        f.write(f"• Profit Target: {PROFIT_TARGET}% (optimized from historical analysis)\n")
        
        f.write("\n" + "="*50 + "\n")
        f.write("PERFORMANCE RESULTS\n")
        f.write("="*50 + "\n")
        
        f.write(f"\nPortfolio Performance:\n")
        f.write(f"• Initial Capital: ${INITIAL_CAPITAL:,}\n")
        f.write(f"• Final Portfolio Value: ${strategy_metrics['final_value']:,.2f}\n")
        f.write(f"• Total Profit: ${strategy_metrics['total_pnl']:,.2f}\n")
        f.write(f"• Total Return: {strategy_metrics['total_return_pct']:.2f}%\n")
        f.write(f"• Annualized Return: {strategy_metrics['annualized_return']*100:.2f}%\n")
        
        f.write(f"\nRisk Metrics:\n")
        f.write(f"• Volatility: {strategy_metrics['volatility']*100:.2f}%\n")
        f.write(f"• Sharpe Ratio: {strategy_metrics['sharpe_ratio']:.3f}\n")
        f.write(f"• Sortino Ratio: {strategy_metrics['sortino_ratio']:.3f}\n")
        f.write(f"• Maximum Drawdown: {strategy_metrics['max_drawdown']*100:.2f}%\n")
        f.write(f"• Calmar Ratio: {strategy_metrics['calmar_ratio']:.3f}\n")
        
        f.write(f"\nTrading Statistics:\n")
        f.write(f"• Total Trades: {len(trades_df):,}\n")
        
        win_trades = trades_df[trades_df['net_pnl'] > 0]
        loss_trades = trades_df[trades_df['net_pnl'] < 0]
        f.write(f"• Win Rate: {len(win_trades)/len(trades_df)*100:.1f}%\n")
        
        if len(win_trades) > 0:
            f.write(f"• Average Win: ${win_trades['net_pnl'].mean():.2f}\n")
        if len(loss_trades) > 0:
            f.write(f"• Average Loss: ${loss_trades['net_pnl'].mean():.2f}\n")
        
        f.write(f"• Largest Win: ${trades_df['net_pnl'].max():.2f}\n")
        f.write(f"• Largest Loss: ${trades_df['net_pnl'].min():.2f}\n")
        
        f.write("\n" + "="*50 + "\n")
        f.write("BENCHMARK COMPARISON\n")
        f.write("="*50 + "\n")
        
        f.write(f"\nComparison vs Buy-and-Hold Strategies:\n")
        comparison_df = analytics.create_benchmark_comparison(strategy_metrics, benchmark_results)
        
        for asset in comparison_df.index:
            asset_name = asset.replace(' Buy-Hold', '')
            total_return = comparison_df.loc[asset, 'total_return_pct']
            sharpe = comparison_df.loc[asset, 'sharpe_ratio']
            max_dd = comparison_df.loc[asset, 'max_drawdown'] * 100
            final_val = comparison_df.loc[asset, 'final_value']
            
            f.write(f"\n{asset_name}:\n")
            f.write(f"  • Total Return: {total_return:.2f}%\n")
            f.write(f"  • Final Value: ${final_val:,.2f}\n")
            f.write(f"  • Sharpe Ratio: {sharpe:.3f}\n")
            f.write(f"  • Max Drawdown: {max_dd:.2f}%\n")
        
        f.write("\n" + "="*50 + "\n")
        f.write("RECOMMENDATIONS\n")
        f.write("="*50 + "\n")
        
        # Generate recommendation based on performance
        returns_ranking = comparison_df['total_return_pct'].sort_values(ascending=False)
        strategy_rank = list(returns_ranking.index).index('FirstBar Strategy') + 1
        
        f.write(f"\nBased on the comprehensive analysis:\n")
        
        if strategy_rank <= 2 and strategy_metrics['sharpe_ratio'] > 0.8:
            f.write("• RECOMMENDATION: STRONG BUY\n")
            f.write("• Strategy demonstrates excellent performance with superior risk-adjusted returns\n")
            f.write("• Consider implementing with appropriate position sizing\n")
        elif strategy_rank <= 3 and strategy_metrics['sharpe_ratio'] > 0.5:
            f.write("• RECOMMENDATION: MODERATE BUY\n")
            f.write("• Strategy shows good performance, monitor risk management closely\n")
            f.write("• Consider smaller position sizes initially\n")
        else:
            f.write("• RECOMMENDATION: FURTHER ANALYSIS REQUIRED\n")
            f.write("• Strategy performance needs improvement or parameter optimization\n")
            f.write("• Consider testing different profit targets or filtering mechanisms\n")
        
        f.write("\n• Risk Management: Implement strict position sizing and daily loss limits\n")
        f.write("• Monitoring: Regular performance review and parameter adjustment\n")
        f.write("• Diversification: Consider multi-asset implementation\n")
        
        f.write("\n" + "="*50 + "\n")
        f.write("DISCLAIMERS\n")
        f.write("="*50 + "\n")
        
        f.write("\n• This analysis is for educational and research purposes only\n")
        f.write("• Past performance does not guarantee future results\n")
        f.write("• All trading involves substantial risk of loss\n")
        f.write("• Consider paper trading before live implementation\n")
        f.write("• Consult with financial advisors before making investment decisions\n")
        
        f.write(f"\n\nReport generated by: {ANALYST_NAME}\n")
        f.write(f"Analysis date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("="*100 + "\n")
    
    print(f"✅ Professional report generated: {report_file}")
    
    # Create a summary report for quick reference
    quick_file = f"../reports/FirstBar_Quick_Summary_{STRATEGY_TICKER}_{timestamp}.txt"
    
    with open(quick_file, 'w') as f:
        f.write(f"FirstBar Strategy - Quick Summary\n")
        f.write(f"Date: {datetime.now().strftime('%Y-%m-%d')}\n")
        f.write(f"Asset: {STRATEGY_TICKER}\n\n")
        
        f.write(f"Performance:\n")
        f.write(f"• Total Return: {strategy_metrics['total_return_pct']:.2f}%\n")
        f.write(f"• Final Value: ${strategy_metrics['final_value']:,.2f}\n")
        f.write(f"• Sharpe Ratio: {strategy_metrics['sharpe_ratio']:.3f}\n")
        f.write(f"• Max Drawdown: {strategy_metrics['max_drawdown']*100:.2f}%\n")
        f.write(f"• Total Trades: {len(trades_df):,}\n")
        
        returns_ranking = comparison_df['total_return_pct'].sort_values(ascending=False)
        strategy_rank = list(returns_ranking.index).index('FirstBar Strategy') + 1
        f.write(f"• Ranking: #{strategy_rank} out of {len(returns_ranking)}\n")
    
    print(f"✅ Quick summary generated: {quick_file}")
    
else:
    print("⚠️ Report generation skipped - insufficient data")

## 🎯 Analysis Completion Summary

In [ ]:
print(f"\n{'🎯'*50}")
print(f"FIRSTBAR FARMING - ANALYSIS COMPLETE")
print(f"{'🎯'*50}")

print(f"\n📊 Analysis Status:")
print(f"   • Data Connection: {'✅ Success' if len(market_data) > 0 else '❌ Failed'}")
print(f"   • Strategy Execution: {'✅ Success' if strategy_success else '❌ Failed'}")
print(f"   • Benchmark Analysis: {'✅ Complete' if benchmark_results else '❌ Incomplete'}")
print(f"   • Visualization: {'✅ Generated' if strategy_success else '❌ Skipped'}")
print(f"   • Report Generation: {'✅ Complete' if strategy_success else '❌ Skipped'}")

if strategy_success:
    print(f"\n🏆 Key Results:")
    print(f"   • Strategy Asset: {STRATEGY_TICKER}")
    print(f"   • Total Return: {strategy_metrics['total_return_pct']:.2f}%")
    print(f"   • Final Portfolio: ${strategy_metrics['final_value']:,.2f}")
    print(f"   • Sharpe Ratio: {strategy_metrics['sharpe_ratio']:.3f}")
    print(f"   • Max Drawdown: {strategy_metrics['max_drawdown']*100:.2f}%")
    print(f"   • Total Trades: {len(trades_df):,}")
    
    if benchmark_results:
        returns_ranking = comparison_df['total_return_pct'].sort_values(ascending=False)
        strategy_rank = list(returns_ranking.index).index('FirstBar Strategy') + 1
        print(f"   • Performance Rank: #{strategy_rank} out of {len(returns_ranking)}")

print(f"\n📁 Output Files:")
print(f"   • Trades: ../results/")
print(f"   • Reports: ../reports/")
print(f"   • Visualizations: Generated in notebook")

print(f"\n⏰ Analysis completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔬 Analyst: {ANALYST_NAME}")

print(f"\n{'='*80}")
print("Thank you for using FirstBar Farming Professional Analysis System")
print(f"{'='*80}")

---

## 📚 Usage Instructions

### Quick Start:
1. **Configure Settings**: Modify the configuration section with your desired ticker and parameters
2. **Run All Cells**: Execute the entire notebook to perform comprehensive analysis
3. **Review Results**: Examine visualizations, performance metrics, and recommendations

### Advanced Usage:
- **Parameter Testing**: Modify `PROFIT_TARGET`, `STRATEGY_MODE`, or `POSITION_SIZE_PCT` for optimization
- **Asset Comparison**: Change `STRATEGY_TICKER` to analyze different assets
- **Period Analysis**: Adjust `DATA_PERIOD` for different historical ranges

### 💾 CSV Data Management:

**Automatic Export**: All IBKR data is automatically saved to CSV files for transferability and reuse.

**CSV File Naming Convention**:
```
../data/ibkr_{symbol}_{duration}_{barsize}_{timestamp}.csv
Example: ../data/ibkr_tqqq_3y_1_hour_20250925_120000.csv
```

**Loading Previously Saved Data**:
```python
# Load most recent CSV for a symbol
data = data_manager.load_from_csv('TQQQ')

# Load specific CSV file
data = data_manager.load_from_csv('TQQQ', '../data/ibkr_tqqq_3y_1_hour_20250925_120000.csv')

# List all saved CSV files
files = data_manager.list_saved_data()
print(files)
```

**Benefits of CSV Export**:
- **Transferability**: Move data between systems and platforms
- **Backup**: Preserve historical data for future analysis
- **Offline Analysis**: Work without live IBKR connection
- **Speed**: Faster loading than re-downloading from IBKR
- **Sharing**: Easy data sharing with team members
- **Compliance**: Maintain data records for audit trails

### Output Files:
- **data/**: Raw IBKR market data in CSV format (automatically saved)
- **results/**: Strategy trades, equity curves, benchmark comparisons
- **reports/**: Professional analysis reports and executive summaries
- **Visualizations**: Generated within notebook for interactive analysis

---

**⚠️ Important Disclaimers:**
- This analysis is for educational and research purposes only
- Past performance does not guarantee future results
- All trading involves substantial risk of loss
- Always paper trade before live implementation
- Consult financial advisors before making investment decisions

**🤖 FirstBar Farming Professional Analysis System**  
*Powered by IBKR TWS API and Advanced Performance Analytics*  
*With Automatic CSV Export for Data Transferability*